In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('../output/categorized_test_statement_section.csv')
test_df

,statement,section,label,InferenceType,SectionType,Type
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction,NLI,Eligibility,Comparison
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment,NLI,Eligibility,Single
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment,NLI,Eligibility,Comparison
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment,NLI,Eligibility,Comparison
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction,Numerical,Intervention,Comparison
...,...,...,...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction,Numerical,Eligibility,Comparison
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment,NLI,Eligibility,Single
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment,Numerical,Intervention,Comparison
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction,Numerical,Results,Single


In [3]:
hypothesis_lst=test_df['statement'].values.tolist()
len(hypothesis_lst)

500

In [4]:
evidence_lst=test_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

500

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

500

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
# text_tok=AutoTokenizer.from_pretrained('roberta-base')
# text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)
#bio-bert
# text_tok=AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
# text_clf=AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1',num_labels=2)
#gpt2
text_tok=AutoTokenizer.from_pretrained('gpt2')
text_tok.pad_token = text_tok.eos_token
text_clf=AutoModelForSequenceClassification.from_pretrained('gpt2',num_labels=2,pad_token_id=text_tok.eos_token_id)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [9]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

tokenize done


In [10]:
scores=[]
# model_names=['bert-base-uncased']+[
#     './output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#roberta-base
# model_names=['roberta-base']+[
#     './output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#biobert
# model_names=['dmis-lab/biobert-v1.1']+[
#     './output/clf_models/biobert-v1.1_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#gpt2
# model_names=['gpt2']+[
#     './output/clf_models/gpt2_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
# for model_name in model_names:

model_name='../output/clf_models/gpt2_epoch_{}.pt'.format(format(3,'05d'))
scores=[]
clf=AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2,pad_token_id=text_tok.eos_token_id).cuda()
with torch.no_grad():
    for batch in range(len(testing_data)):
        batch_texts,batch_labels=testing_data[batch]
        scores.append(F.softmax(clf(**batch_texts).logits,dim=1).detach().cpu().numpy())
        print('model:',model_name,'batch:',batch,end='\r')
scores=np.concatenate(scores,axis=0)
_=clf.cpu()

In [11]:
df2=test_df.copy()
df2['score']=scores[:,1]
df2

,statement,section,label,InferenceType,SectionType,Type,score
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction,NLI,Eligibility,Comparison,0.411579
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment,NLI,Eligibility,Single,0.618410
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment,NLI,Eligibility,Comparison,0.595187
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment,NLI,Eligibility,Comparison,0.545104
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction,Numerical,Intervention,Comparison,0.510713
...,...,...,...,...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction,Numerical,Eligibility,Comparison,0.442697
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment,NLI,Eligibility,Single,0.432401
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment,Numerical,Intervention,Comparison,0.500187
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction,Numerical,Results,Single,0.524062


In [12]:
df2.to_csv('../output/categorized_gpt2_score.csv',index=False)